# Library

In [20]:
import requests
from bs4 import BeautifulSoup as bs

import time, os
import pandas as pd
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [21]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [22]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [23]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [24]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

# 페이지 접속

In [25]:
# 페이지 주소
url = 'https://news.naver.com/section/101' # 네이버 뉴스 > 경제

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [26]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

## selenium 작동

In [27]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

## 헤드라인 뉴스 섹션 로드

In [28]:
# 헤드라인 뉴스
headline_news_more_view_button_xpath = '//*[@id="newsct"]/div[1]/div[2]/a' # 헤드라인 더 보기 버튼

# 헤드라인 더 보기 버튼 로딩 대기
headline_news_more_view_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, headline_news_more_view_button_xpath))
)

# 헤드라인 더보기 버튼 클릭
headline_news_more_view_button.click()
driver.implicitly_wait(10)

# 헤드라인 뉴스 수집

In [29]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'경제 : 네이버 뉴스'

In [30]:
# 헤드라인 뉴스 섹션
headline_news_section = soup.select_one('ul.sa_list')
len(headline_news_section)

21

In [31]:
# 뉴스 타이들 리스트
headline_news_list = headline_news_section.select('li.sa_item._SECTION_HEADLINE')
len(headline_news_list)

10

# 수집한 데이터 전처리

In [38]:
link_list = []
title_list = []
content_list = []
press_list = []
related_news_count_list = []

for i in range(len(headline_news_list)):
    # 기사 내용 섹션
    news_content_section = headline_news_list[i].select_one('div.sa_text')

    # 기사 링크
    try:
        link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
    except:
        link = np.nan
        print(i)
        # print('기사 링크 없음')
    
    # 타이틀
    try:
        title = news_content_section.select_one('strong.sa_text_strong').text
    except:
        title = np.nan
        print(i)
        # print('타이틀 없음')
    
    # 기사 내용
    try:
        content = news_content_section.select_one('div.sa_text_lede').text
    except:
        content = np.nan
        print(i)
        # print('기사 내용 없음')
    
    ## 언론사
    try:
        press = news_content_section.select_one('div.sa_text_press').text
    except:
        press = np.nan
        print(i)
        # print('언론사 없음')
    
    ## 관련 뉴스 개수
    try:
        related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
    except:
        related_news_count = np.nan
        print(i)
        # print('관련 뉴스 개수 없음')
    
    # 정보 리스트에 담기
    link_list.append(link)
    title_list.append(title)
    content_list.append(content)
    press_list.append(press)
    related_news_count_list.append(related_news_count)

In [39]:
len(link_list), len(title_list), len(content_list), len(press_list), len(related_news_count_list)

(10, 10, 10, 10, 10)

In [40]:
data_dict = {
    '번호': range(1, len(link_list)+1),
    '기사 링크': link_list,
    '기사 제목': title_list,
    '기사 내용': content_list,
    '언론사': press_list,
    '관련 뉴스 개수': related_news_count_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,기사 링크,기사 제목,기사 내용,언론사,관련 뉴스 개수
0,1,https://n.news.naver.com/mnews/article/025/000...,"트럼프 만난 정용진 “韓 저력 믿고 기다려 달라 했다""…머스크와도 인사",“(트럼프 측근들과 만나) 대한민국은 저력이 있는 나라니 믿고 기다려달라고 했다.”...,중앙일보,110
1,2,https://n.news.naver.com/mnews/article/056/001...,“편법 증여·환치기”…외국인 부동산 불법 거래 무더기 적발,외국인이 부동산을 부정한 방법으로 거래한 것으로 의심되는 사례가 무더기로 적발됐습니...,KBS,42
2,3,https://n.news.naver.com/mnews/article/008/000...,"HD현대오일뱅크, 세계 최대 액침냉각 기업서 냉각유 인증 획득",HD현대오일뱅크가 데이터센터 열을 식혀주는 액침냉각유 시장 공략에 나선다고 22일 ...,머니투데이,36
3,4,https://n.news.naver.com/mnews/article/119/000...,금융사 개인사업자 대출 비교 공시 시작된다,금융위원회는 각 6개 금융협회·중앙회와 함께 오는 23일부터 개인사업자 대출상품 비...,데일리안,17
4,5,https://n.news.naver.com/mnews/article/366/000...,한경협 “내년 수출 1.4% 증가 그친다…자동차·철강은 감소 전망”,한국경제인협회(한경협)는 22일 내년 12대 수출 주력 업종의 수출 증가율이 1.4...,조선비즈,48


# 데이터 출력

In [41]:
current_date = time.strftime('%Y%m%d')
current_date

'20241222'

In [42]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241222_PM_114037'

In [43]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [ ]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_news', 'economy', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\naver_news\\20241222'

In [45]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [46]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_news_economy_{current_datetime}.xlsx'), index=False)

In [ ]:
driver.quit()
print('수집 완료')

# END